# Classifying text lines

A classifier is trained to classify single text lines of a report. For example, the lines can be classified as containing a diagnosis (`"dm"`) or history (`his`) of a patient. This task was used as a preprocessing step to later steps of structured information extraction. So that only lines classified as containing a diagnosis will be fed to a downstream classifier, extracting the exact diagnosis. This step might be unnecessary with modern transformers that can handle longer text inputs.

The files containing the necessary information are inside the `data/raw/labelling` directory. It contains manually labelled reports from different sessions.

The original classes per label are:

| category    | subcategory       | abbreviation |
|-------------|-------------------|--------------|
| diagnosis   | MS diagnosis      | dm           |
|             | other             | do           |
| current state     |              | cu           |
| history     |                   | his          |
| symptoms    | MS related        | sym          |
|             | other             | so           |
| MRI | results                  | mr           |
| lab | results                  | labr         |
|             | other             | labo         |
| medication  | MS related        | medms        |
|             | other             | medo         |
| test, treatment        | results | tr           |
| header      |                   | head         |
| unknown     |                   | unk          |

I will group the classes according to the original approach, and drop observations with no text or label.

In [1]:
import sys
import os
sys.path.append(os.getcwd()+"/../..")
from src import paths
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import json

In [2]:
def load_line_labelling():
    """Loading the data from the nested csv files in the different "imported_time" directories. Labelled reports have a "rev.csv" ending
    and are in a "_Marc" subdirectory. There should only be one entry per rid, that is labelled. Duplicates will be removed.

    Returns:
        pd.DataFrame: Dataframe with columns: "text", "class", "rid"
    """
    df_list = []
    rid_list = []

    for root, dirs, files in os.walk(paths.DATA_PATH_LABELLED):
        if "Marc" not in root:
            continue
        for file in files:

            # Get the research id from filename
            rid = file.split("_")[0]
            
            # Check if the file is a labelled report (and not mri) and if the rid is already in the list
            if (file.endswith("rev.csv") and "mri" not in file and rid not in rid_list):
                
                # Append rid to rid list to keep track of which files have been added
                rid_list.append(rid)

                # Create a dataframe from the csv file
                _df = pd.read_csv(os.path.join(root, file))
                
                # Add the rid to the dataframe
                _df = _df.rename(columns={"text": "text", "class": "class"})
                _df['rid'] = rid
                
                # Append the dataframe to the main dataframe
                try: 
                    df_list.append(_df)
                except:
                    print("Error with file: ", file)
                    print("df head: ", _df.head(5))
                    print("_df head: ", _df.head(5))
                    continue
    print("Number of reports: ", len(df_list))
    return pd.concat(df_list)[["text", "class", "rid"]]

In [3]:
def clean_line_text(df: pd.DataFrame):
    """
    For Transformer to work, there has to be text in the text column. If there is no text, the text column is removed.
    """
    df = df.dropna(subset=["text"])

    return df

def clean_line_class(df: pd.DataFrame):
    """Cleans the dataframe labels in "class".
    1) Removes whitespace from beginning and end of text
    2) Correct spelling mistakes
    3) Fill NaN values with "unk"
    4) Create a new column "class_agg" with the aggregated classes of the original approach.

    Args:
        df (pd.DataFrame): Input dataframe
    """

    # Class mapping spelling mistakes
    class_mapping_spelling = {
        'memds': 'medms',
        'hs': 'his',
        'm': 'mr',
    }

    # Classes of original approach abbreviation
    # classes_orig = ["dm", "do", "cu", "his", "sym", "so", "mr", "labr", "labo", "medms", "medo", "tr", "head", "unk"]

    # Class mapping of original approach
    # df = df.assign(class2 = df['class'])
    # df.loc[(df['class'] == 'his') | (df['class'] == 'sym') | (df['class'] == 'cu'), 'class2'] = 'his_sym_cu'
    # df.loc[(df['class'] == 'labr') | (df['class'] == 'labo'), 'class2'] = 'labr_labo'
    # df.loc[(df['class'] == 'to') | (df['class'] == 'tr'), 'class2'] = 'to_tr'
    # df.loc[(df['class'] == 'medo') | (df['class'] == 'unk') | (df['class'] == 'do') | (df['class'] == 'so'), 'class2'] = 'medo_unk_do_so'
    
    class_mapping_agg = {
        'his': 'his_sym_cu',
        'sym': 'his_sym_cu',
        'cu': 'his_sym_cu',
        'labr': 'labr_labo',
        'labo': 'labr_labo',
        'to': 'to_tr',
        'tr': 'to_tr',
        'medo': 'medo_unk_do_so',
        'unk': 'medo_unk_do_so',
        'do': 'medo_unk_do_so',
        'so': 'medo_unk_do_so',
    }

    # Cleaning the class column
    df['class'] = df['class'].str.strip() \
                             .replace(class_mapping_spelling) \
                             .fillna("unk") 

    # Creating a new column with the aggregated classes
    df['class_agg'] = df['class'].replace(class_mapping_agg)

    return df

def load_clean_line_df():
    """Loads and cleans the dataframe from the load_line_labelling function.
    """
    df = load_line_labelling()
    df = clean_line_text(df)
    df = clean_line_class(df)

    return df[["rid", "text", "class", "class_agg"]]

In [4]:
df = load_clean_line_df()
display(df.head(5))

# Class distribution
print("class_agg values: ", df.class_agg.value_counts())
print("class values: ", df["class"].value_counts())

# Number of reports with missing values in class_agg
print("Number of reports with missing values in class_agg: ", df[df["class_agg"].isnull()].shape[0])

Number of reports:  74


,rid,text,class,class_agg
0,014673E3-8EBF-4408-96E0-79BE34341EB5,"Schubförmige Multiple Sklerose (RRMS), EDSS 3.5",dm,dm
1,014673E3-8EBF-4408-96E0-79BE34341EB5,"(ES 04/1992, ED 07/1998 (USZ))",do,medo_unk_do_so
2,014673E3-8EBF-4408-96E0-79BE34341EB5,Verlauf:,head,head
3,014673E3-8EBF-4408-96E0-79BE34341EB5,"INDENT 04/1992 Kribbelparästhesien bd. Füsse, ...",his,his_sym_cu
4,014673E3-8EBF-4408-96E0-79BE34341EB5,INDENT 1997 Fühl- und auch Kraftminderung des ...,his,his_sym_cu


class_agg values:  class_agg
his_sym_cu        312
mr                270
head              236
medms             213
labr_labo         184
medo_unk_do_so    139
to_tr             114
dm                 74
Name: count, dtype: int64
class values:  class
mr       270
head     236
medms    213
his      210
labr     166
tr        77
dm        74
sym       62
unk       50
do        42
cu        40
to        37
medo      35
labo      18
so        12
Name: count, dtype: int64
Number of reports with missing values in class_agg:  0


In [5]:
# Save the dataframe
df.to_csv(os.path.join(paths.DATA_PATH_PREPROCESSED, "line_labelling/line_labelling_clean.csv"))

In [6]:
# Create a train test split by rid
rids = df.rid.unique()
rids_train, rids_test = train_test_split(rids, test_size=0.2, random_state=42)
rids_train, rids_val = train_test_split(rids_train, test_size=0.2, random_state=42)

df_train = df[df.rid.isin(rids_train)]
df_val = df[df.rid.isin(rids_val)]
df_test = df[df.rid.isin(rids_test)]

# Save the train, val and test dataframes
df_train.to_csv(os.path.join(paths.DATA_PATH_PREPROCESSED, "line_labelling/line_labelling_clean_train.csv"))

df_val.to_csv(os.path.join(paths.DATA_PATH_PREPROCESSED, "line_labelling/line_labelling_clean_val.csv"))

df_test.to_csv(os.path.join(paths.DATA_PATH_PREPROCESSED, "line_labelling/line_labelling_clean_test.csv"))

In [13]:
# Create HuggingFace Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "val": Dataset.from_pandas(df_val),
    "test": Dataset.from_pandas(df_test),
})

# Drop __index_level_0__ column
dataset = dataset.remove_columns("__index_level_0__")

# Encode the labels
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
enc.fit(np.stack(dataset["train"]["class_agg"]).reshape(-1, 1))
dataset = dataset.map(lambda e: {"labels": enc.transform(np.stack(e["class_agg"]).reshape(-1, 1)).astype(int)}, batched=True)

# Save the dataset
dataset.save_to_disk(os.path.join(paths.DATA_PATH_PREPROCESSED, "line_labelling/line_labelling_clean_dataset"))

# Save the class mapping in a json file
class_mapping = {i: c for i, c in enumerate(enc.categories_[0])}
with open(os.path.join(paths.DATA_PATH_PREPROCESSED, "line_labelling/line_labelling_clean_class_mapping.json"), 'w') as fp:
    json.dump(class_mapping, fp)

Map:   0%|          | 0/948 [00:00<?, ? examples/s]

Map:   0%|          | 0/249 [00:00<?, ? examples/s]

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/948 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/249 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/345 [00:00<?, ? examples/s]